<a href="https://colab.research.google.com/github/trong-shen/Game-of-Throne-Project/blob/master/GOT_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load the CSV file from Github

In [12]:
import pandas as pd
import numpy as np

GOT= pd.read_csv('https://raw.githubusercontent.com/trong-shen/Game-of-Throne-Project/master/Game_of_Thrones_Script_clean.csv')

#Extract only season one data and two data
GOT1=GOT[GOT.Season=="Season 1"]
GOT2=GOT[GOT.Season=="Season 2"]

print(GOT1.head())
print(GOT1.info())

print(GOT2.head())
print(GOT1.info())

  Release Date  ...                                           Sentence
0    4/17/2011  ...  What do you expect? They're savages. One lot s...
1    4/17/2011  ...  I've never seen wildlings do a thing like this...
2    4/17/2011  ...                             How close did you get?
3    4/17/2011  ...                            Close as any man would.
4    4/17/2011  ...                   We should head back to the wall.

[5 rows x 6 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3179 entries, 0 to 3178
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Release Date   3179 non-null   object
 1   Season         3179 non-null   object
 2   Episode        3179 non-null   object
 3   Episode Title  3179 non-null   object
 4   Name           3179 non-null   object
 5   Sentence       3179 non-null   object
dtypes: object(6)
memory usage: 173.9+ KB
None
     Release Date  ...                            Sente

In [13]:
# Keep character lines if and only if they still exist in season 2

#Find a unique list of characters in season 2

char_S2=GOT2.Name.unique()
print(len(char_S2))

#Filter S1 data if characters are in S2
GOT1_modified=GOT1[GOT1['Name'].isin(char_S2)]
print(len(GOT1_modified.Name.unique()))
print(GOT1_modified.Name.unique())

136
49
['jon snow' 'sansa stark' 'robb stark' 'catelyn stark' 'bran stark'
 'theon greyjoy' 'jaime lannister' 'cersei lannister' 'luwin' 'arya stark'
 'tyrion lannister' 'ros' 'daenerys targaryen' 'jorah mormont'
 'khal drogo' 'sandor clegane' 'doreah' 'irri' 'joffrey lannister'
 'myrcella baratheon' 'rodrick cassel' 'soldier' 'varys' 'renly baratheon'
 'petyr baelish' 'grand maester pycelle' 'guard' 'jeor mormont' 'grenn'
 'lancel lannister' 'rakharo' 'yoren' 'sam tarly' 'janos'
 'gendry baratheon' 'bronn' 'loras tyrell' 'osha' 'wildling' 'man'
 'tywin lannister' 'meryn trant' 'kevan lannister' 'all' 'prostitute'
 'shae' 'rickon stark' 'karstark' 'hot pie']


In [15]:

#Further filter based on characters of interest 

important_names=np.delete(GOT1_modified.Name.unique(),21);
important_names=np.delete(important_names,25);
print(important_names)


print(len(important_names))

GOT1_modified=GOT1_modified[GOT1_modified['Name'].isin(important_names)]
GOT1_modified.head()
print(len(GOT1_modified.Name.unique()))

GOT1_modified.info()


['jon snow' 'sansa stark' 'robb stark' 'catelyn stark' 'bran stark'
 'theon greyjoy' 'jaime lannister' 'cersei lannister' 'luwin' 'arya stark'
 'tyrion lannister' 'ros' 'daenerys targaryen' 'jorah mormont'
 'khal drogo' 'sandor clegane' 'doreah' 'irri' 'joffrey lannister'
 'myrcella baratheon' 'rodrick cassel' 'varys' 'renly baratheon'
 'petyr baelish' 'grand maester pycelle' 'jeor mormont' 'grenn'
 'lancel lannister' 'rakharo' 'yoren' 'sam tarly' 'janos'
 'gendry baratheon' 'bronn' 'loras tyrell' 'osha' 'wildling' 'man'
 'tywin lannister' 'meryn trant' 'kevan lannister' 'all' 'prostitute'
 'shae' 'rickon stark' 'karstark' 'hot pie']
47
47
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2134 entries, 15 to 3178
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Release Date   2134 non-null   object
 1   Season         2134 non-null   object
 2   Episode        2134 non-null   object
 3   Episode Title  2134 non-n

In [0]:
!pip install nltk
import nltk
nltk.download('all')


In [0]:
#Tokenize the words and remove between words punctunations
tokenizer=nltk.RegexpTokenizer(r"\w+")
GOT1['Tokenized_Sentence']=GOT1.Sentence.apply(lambda x:tokenizer.tokenize(x.lower()))
GOT1.head()


In [0]:
#Remove stopwords for sentiment analysis
stopword=nltk.corpus.stopwords.words('english')


In [0]:
#Define a function to remove stop words
def remove_stopwords(tokenized_sentence):
  text=[word for word in tokenized_sentence if word not in stopword]
  return text

In [0]:
#Implement the stop words function to a new column 
GOT1['Tokenized_No_Stop']=GOT1.Tokenized_Sentence.apply(lambda x:remove_stopwords(x))
GOT1.head()

In [0]:
# Stemming of the Non Stop Words Column
wn=nltk.WordNetLemmatizer()

def stem_reduction(tokenized_sentence):
  sentence=[wn.lemmatize(word) for word in tokenized_sentence]
  return (sentence)

GOT1['Stemmed_Sentence']=GOT1['Tokenized_No_Stop'].apply(lambda x:stem_reduction(x))
GOT1.tail()